# Validación de Reglas de Calidad (QC Rules)
Este notebook implementa el bloque `validate_qc_rules` del pipeline.

Valida reglas estructurales y de negocio sobre el corpus normalizado para asegurar su consistencia antes de exportarlo a etapas posteriores (índices RAG, GUI, evaluación).

## 1) Carga del corpus
Se intentará cargar primero el **corpus normalizado** (`corpus_cmv_normalized.csv`). Si no existe, se usará el **corpus base** (`corpus_cmv.csv`).

In [2]:

import pandas as pd
from pathlib import Path

BASE_DIR = Path("../../data_curated/corpus_estructurado/")
CSV_PATH = BASE_DIR / "corpus_cmv.csv"


df = pd.read_csv(CSV_PATH, dtype=str)
print("Archivo cargado:", CSV_PATH)
print(f"Registros: {len(df)}, Columnas: {df.shape[1]}")
df.head(2)


Archivo cargado: ../../data_curated/corpus_estructurado/corpus_cmv.csv
Registros: 2, Columnas: 281


,file,CP,CPP,DO,VERSION,ID,RUC,LOGO_PATH,NOMBRE_ENTID_CONTRAT,PROVINCIA_COD,...,FECHA_RECEPCION_NORMALIZADA,TIPO_FECHA_NORMALIZADA,FECHA_NACIMIENTO_CP_NORMALIZADA,FECHA_GRADUACION_CP_NORMALIZADA,FECHA_OBTEN_TITUL_CP_NORMALIZADA,PRESUPUESTO_REFERENCIAL_NUMEROS_USD,VALOR_OFERTADO_USD,VALOR_CONTRATO_USD,TOTAL_USD,ANIO
0,oferta_pli_cotobr.ushay,d49a8f53c5feb3a8077827b9d2a8a54c,COTO-EPN-029-2023,1001581055001,1.6.0,8,1760005620001,1760005620001principal_color.jpg,ESCUELA POLITÉCNICA NACIONAL,17,...,2020-01-14,NaN,1986-08-26,2012-03-26,2012-03-26,493395.94,501.49,41402.28,100.0,2023
1,contratacion_pli_cotobr.ushay,d9f4df1b886038572eb272e5d62c45a4,NaN,NaN,1.2,14,0960006180001,escudo.jpg,GAD MUNICIPAL DEL CANTON NOBOL,09,...,NaN,NaN,NaN,NaN,NaN,495930.26,NaN,NaN,100.0,2023


## 2) Configuración de reglas
Define umbrales y valores esperados para cada validación.

In [3]:

RULES = {
    "mandatory_columns": [
        "COD_PROC", "NOMBRE_ENTID_CONTRAT", "RUC"
    ],
    "min_non_null_ratio": 0.7,
    "unique_keys": ["COD_PROC"],
    "ruc_length": 13,
    "date_suffix": "_NORMALIZADA",
    "numeric_fields": [
        "PRESUPUESTO_REFERENCIAL_NUMEROS_USD",
        "VALOR_CONTRATO_USD",
        "VALOR_OFERTADO_USD",
        "TOTAL_USD"
    ],
    "categorical_maps": {
        "TIPO_PERSONA_NORMALIZADA": ["Natural", "Jurídica"]
    }
}
RULES


{'mandatory_columns': ['COD_PROC', 'NOMBRE_ENTID_CONTRAT', 'RUC'],
 'min_non_null_ratio': 0.7,
 'unique_keys': ['COD_PROC'],
 'ruc_length': 13,
 'date_suffix': '_NORMALIZADA',
 'numeric_fields': ['PRESUPUESTO_REFERENCIAL_NUMEROS_USD',
  'VALOR_CONTRATO_USD',
  'VALOR_OFERTADO_USD',
  'TOTAL_USD'],
 'categorical_maps': {'TIPO_PERSONA_NORMALIZADA': ['Natural', 'Jurídica']}}

## 3) Utilidades de validación

In [4]:

import numpy as np
import pandas as pd

def ratio_non_null(series):
    return float(series.notna().mean()) if len(series) else 0.0

def is_numeric_series(series):
    try:
        pd.to_numeric(series.dropna(), errors="raise")
        return True
    except Exception:
        return False

def validate_mandatory_columns(df, cols, min_ratio):
    results = []
    for c in cols:
        exists = c in df.columns
        ratio = ratio_non_null(df[c]) if exists else 0.0
        passed = exists and (ratio >= min_ratio)
        results.append({
            "rule": "mandatory_columns",
            "column": c,
            "exists": exists,
            "non_null_ratio": round(ratio, 3),
            "threshold": min_ratio,
            "passed": passed
        })
    return results

def validate_uniqueness(df, keys):
    results = []
    for k in keys:
        if k in df.columns:
            dupes = df.duplicated(subset=[k]).sum()
            passed = (dupes == 0)
        else:
            dupes = None
            passed = False
        results.append({
            "rule": "unique_key",
            "column": k,
            "duplicates": dupes,
            "passed": passed
        })
    return results

def validate_ruc(df, col="RUC", expected_len=13):
    exists = col in df.columns
    if not exists:
        return [{
            "rule": "ruc_length",
            "column": col,
            "exists": False,
            "valid_count": 0,
            "total": len(df),
            "passed": False
        }]
    ser = df[col].astype(str).str.strip()
    valid = ser.notna() & ser.str.fullmatch(r"\d{"+str(expected_len)+r"}")
    return [{
        "rule": "ruc_length",
        "column": col,
        "exists": True,
        "valid_count": int(valid.sum()),
        "total": len(df),
        "passed": bool(valid.sum() == ser.notna().sum())
    }]

def validate_date_norm(df, suffix="_NORMALIZADA"):
    results = []
    date_cols = [c for c in df.columns if c.upper().startswith("FECHA") or c.endswith(suffix) and not 'PERSONA' in c.upper()]
    for c in date_cols:
        if c not in df.columns:
            continue
        ser = df[c]
        if c.endswith(suffix):
            pat = ser.dropna().str.fullmatch(r"\d{4}-\d{2}-\d{2}")
            passed = bool(pat.all()) if len(pat) else True
            results.append({
                "rule": "date_iso_format",
                "column": c,
                "pattern": "YYYY-MM-DD",
                "checked_rows": int(ser.notna().sum()),
                "passed": passed
            })
        else:
            results.append({
                "rule": "date_raw_presence",
                "column": c,
                "non_null_ratio": round(ratio_non_null(ser), 3),
                "passed": True
            })
    return results

def validate_numeric(df, fields):
    results = []
    for c in fields:
        exists = c in df.columns
        if not exists:
            results.append({
                "rule": "numeric_field",
                "column": c,
                "exists": False,
                "is_numeric": False,
                "passed": False
            })
            continue
        ser = df[c]
        try:
            pd.to_numeric(ser.dropna(), errors="raise")
            numeric_ok = True
        except Exception:
            numeric_ok = False
        results.append({
            "rule": "numeric_field",
            "column": c,
            "exists": True,
            "is_numeric": numeric_ok,
            "passed": numeric_ok
        })
    return results

def validate_categorical(df, mapping):
    results = []
    for col, allowed in mapping.items():
        exists = col in df.columns
        if not exists:
            results.append({
                "rule": "categorical_allowed",
                "column": col,
                "exists": False,
                "invalid_values": None,
                "passed": False
            })
            continue
        ser = df[col].dropna().astype(str).str.strip()
        invalid = sorted(set(ser.unique()) - set(allowed))
        results.append({
            "rule": "categorical_allowed",
            "column": col,
            "exists": True,
            "invalid_values": invalid,
            "passed": (len(invalid) == 0)
        })
    return results


## 4) Ejecutar validaciones

In [5]:

results = []
results += validate_mandatory_columns(df, RULES["mandatory_columns"], RULES["min_non_null_ratio"])
results += validate_uniqueness(df, RULES["unique_keys"])
results += validate_ruc(df, "RUC", RULES["ruc_length"])
results += validate_date_norm(df, RULES["date_suffix"])
results += validate_numeric(df, RULES["numeric_fields"])
results += validate_categorical(df, RULES["categorical_maps"])

import pandas as pd
report_df = pd.DataFrame(results)
report_df


,rule,column,exists,non_null_ratio,threshold,passed,duplicates,valid_count,total,pattern,checked_rows,is_numeric,invalid_values
0,mandatory_columns,COD_PROC,True,1.0,0.7,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mandatory_columns,NOMBRE_ENTID_CONTRAT,True,1.0,0.7,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mandatory_columns,RUC,True,1.0,0.7,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,unique_key,COD_PROC,NaN,NaN,NaN,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ruc_length,RUC,True,NaN,NaN,True,NaN,2.0,2.0,NaN,NaN,NaN,NaN
5,date_raw_presence,FECHA,NaN,1.0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,date_raw_presence,FECHA_FIRMA_OFERTA,NaN,0.5,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,date_raw_presence,FECHA_FABRICACION,NaN,0.5,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,date_raw_presence,FECHA_RECEPCION,NaN,0.5,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,date_raw_presence,FECHA_NACIMIENTO_CP,NaN,0.5,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 5) Resumen ejecutivo

In [6]:

passed_ratio = report_df["passed"].mean()
total_rules = len(report_df)
passed_rules = int(report_df["passed"].sum())
print(f"Reglas cumplidas: {passed_rules}/{total_rules} ({passed_ratio*100:.1f}%)")


Reglas cumplidas: 27/28 (96.4%)


## 6) Exportar reporte

In [10]:

from pathlib import Path
import json

OUT_DIR = Path("../../docs/validations")
OUT_DIR.mkdir(parents=True, exist_ok=True)

report_md  = OUT_DIR / "validation_report.md"
report_json = OUT_DIR / "validation_summary.json"

md = ["# Validation Report", "", f"Archivo: {CSV_PATH}", ""]
md.append(report_df.to_markdown(index=False))
report_md.write_text("\n".join(md), encoding="utf-8")

summary = {
    "file": str(CSV_PATH),
    "total_rules": int(total_rules),
    "passed_rules": int(passed_rules),
    "passed_ratio": float(round(passed_ratio, 3))
}
report_json.write_text(json.dumps(summary, indent=2, ensure_ascii=False), encoding="utf-8")

print("Exportado:")
print(" -", report_md)
print(" -", report_json)


Exportado:
 - ../../docs/validations/validation_report.md
 - ../../docs/validations/validation_summary.json
